# Analyze the clusters w.r.t. the spectrum identifications

In [1]:
import pandas as pd
import math
import numpy as np

from collections import Counter
import operator
from tqdm.notebook import tqdm

import bittremieux_utils
from spectrum_utils.spectrum import MsmsSpectrum
from ms_io import ms_io

import spectrum_utils.plot as sup
import spectrum_utils.spectrum as sus
import matplotlib.pyplot as plt
from os import path

### Import the clustering file and load the identifications

In [ ]:
file_labels = '/media/maesk/WD/MS/PXD000561/kim2014_ids.csv'
labels = pd.read_csv(file_labels)
labels['sequence'] = labels['sequence'].str.replace('L', 'I')

cluster_labels = bittremieux_utils.get_clusters_falcon(
    '/media/maesk/WD/falcon/PXD000561/nn/' + \
        'fragm_tol_0.05_hash_len_800/prec_tol_20/clusters_eps_0.1_minsample_2.csv',
    labels
)

# Extract metadata from identifier
development = []
tissue = []
spectrometer = []

for id in cluster_labels['identifier'].tolist():
    _, _, id, _, _ = id.split(':')
    d, t, _, ms, _, _ = id.split('_')

    development.append(d)
    tissue.append(t)
    spectrometer.append(ms)

cluster_labels.insert(0, 'development', development)
cluster_labels.insert(1, 'spectrometer', spectrometer)
cluster_labels.insert(2, 'tissue', tissue)

cluster_labels.sort_values(by='cluster', inplace=True)
cluster_labels.reset_index(drop=True, inplace=True)
cluster_labels.to_csv('/media/maesk/WD/test.csv')

### Functions to analyze/classify the clusters

In [ ]:
def getMax(counter):
    maj_key = max(counter.items(), key=operator.itemgetter(1))[0]
    return maj_key, counter[maj_key]

def clusterType(ids):
    nsp_cl = len(ids)
    count = Counter(ids)

    # Get the key with larger count
    maj_key, _ = getMax(count)
    nlabeled = len([id for id in ids if not pd.isna(id)])
    if len(count) == 1:
        maj_identified = 0
    else:
        _, maj_identified = getMax(Counter([id for id in ids if not pd.isna(id)]))

    if (count[maj_key] > 0.7*nsp_cl) and not pd.isna(maj_key):
        type = 'identified'
    elif count[maj_key] == nsp_cl and pd.isna(maj_key):
        type = 'unidentified'
    elif maj_identified > 0.7*nlabeled: # Most identifications are similar
        type = 'coherent'
    else:
        type = 'incoherent'

    return type, nlabeled/nsp_cl

def majTissue(tissues):
    nsp_cl = len(tissues)
    count = Counter(tissues)
    maj_tissue, cnt = getMax(count)
    return maj_tissue, cnt/nsp_cl


def addInterferences(interferences, sequences):
    sequences = sequences.copy()
    sequences = [s for s in sequences if not pd.isna(s)]
    sequences = [s.split('/')[0] for s in sequences]
    count = Counter(sequences)
    for seq1, n1 in count.items():
        for seq2, n2 in count.items():
            if seq2 > seq1:
                if seq1 not in interferences:
                    interferences[seq1] = {}

                if seq2 not in interferences[seq1]:
                    interferences[seq1][seq2] = 0

                interferences[seq1][seq2] = interferences[seq1][seq2] + n1*n2

### Make a summary with the info for each cluster

In [ ]:
nclusters = max(cluster_labels['cluster'])
clids = []
cltypes = []
clsizes = []
clproplabeleds = []
maj_tissues = []
prop_maj_tissues = []

interferences = {}

curr_cluster = 0

clids = []
curr_sequences = []
curr_tissues = []
curr_size = 0

# Iterate over the dataframe
for index, row in tqdm(cluster_labels.iterrows()):
    if row['cluster'] == -1:
        continue

    if curr_cluster != row['cluster']:
        cltype, prop = clusterType(curr_sequences)
        cltypes.append(cltype)
        clproplabeleds.append(prop)
        clsizes.append(curr_size)
        maj_tissue, prop_maj_tissue = majTissue(curr_tissues)
        maj_tissues.append(maj_tissue)
        prop_maj_tissues.append(prop_maj_tissue)
        addInterferences(interferences, curr_sequences)

        clids.append(curr_cluster)
        curr_sequences = []
        curr_tissues = []
        curr_size = 0
        curr_cluster = curr_cluster + 1

        assert curr_cluster == row['cluster'] # Clusters should be ordered

    curr_sequences.append(row['sequence'])
    curr_tissues.append(row['tissue'])
    curr_size = curr_size + 1

cl_summary = pd.DataFrame({'id': clids, 'type':cltypes, 'size': clsizes,
                           'prop_clustered': clproplabeleds,
                           'maj_tissue': maj_tissues, 'prop_maj_tissue': prop_maj_tissues
                           })
cl_summary.sort_values(by='size', ascending=False).to_csv('summary_PXD0000561.csv')

### Plot the different "types" of clusters

In [ ]:
bins = [2, 3, 5, 10, 20, 20000]

labels = []
sizes = {'unidentified': [], 'coherent': [],
         'identified': [], 'incoherent': []}

for i in range(0, len(bins)-1):
    bottom, top = bins[i], bins[i+1]
    labels.append('[%d,%d[' % (bottom, top))
    for type, size in sizes.items():
        mask1 = cl_summary['type'] == type
        mask2 = (cl_summary['size'] >= bottom) & (cl_summary['size'] < top)
        sizes[type].append(len(cl_summary[mask1 & mask2]))

x = np.arange(len(labels))
width = 0.20

fig, axs = plt.subplots(1,2, figsize=(10,4))
b1 = axs[0].bar(x-width*1.5, sizes['identified'], width, label='Identified')
b2 = axs[0].bar(x-width/2, sizes['unidentified'], width, label='Unidentified')
b3 = axs[0].bar(x+width/2, sizes['coherent'], width, label='Coherent')
b4 = axs[0].bar(x+width*1.5, sizes['incoherent'], width, label='Incoherent')

axs[0].set_xlabel('Cluster size')
axs[0].set_ylabel('Number of clusters')
axs[0].set_title('Cluster sizes by cluster type')
axs[0].set_xticks(x)
axs[0].set_xticklabels(labels)
axs[0].legend()

labels = ['identified', 'unidentified', 'coherent', 'incoherent']
axs[1].pie([sum(sizes[l]) for l in labels], labels=[l.capitalize() for l in labels], autopct='%1.1f%%')
axs[1].set_title('Cluster types')

plt.tight_layout()
plt.savefig('results_0.1/cluster_sizes.png', dpi=300)

In [ ]:
print(len(cl_summary[(cl_summary['prop_maj_tissue'] == 1) & (cl_summary['type'] == 'unidentified')])/len(cl_summary))
print(len(cl_summary[cl_summary['type'] == 'unidentified']))

### Show interferences as graph

In [ ]:
# Create a dataframe from the dic
seq1 = []
seq2 = []
nintf = []

for s1, d1 in interferences.items():
    for s2, c2 in d1.items():
        seq1.append(s1)
        seq2.append(s2)
        nintf.append(c2)

interferences_df = pd.DataFrame({'seq1': seq1, 'seq2': seq2, 'count': nintf})
interferences_df.sort_values(by='count', ascending=False, inplace=True)
interferences_df.to_csv('results_0.1/interferences.csv')
interferences_df.head()



### Function to generate representative spectrum

In [ ]:
def representative_spectrum(sps, consensus_identifier, min_intensity=0.01, max_num_peaks=1000, tol=0.05):
    nsp = len(sps)

    # Process the spectra
    sps = [sp.filter_intensity(min_intensity, max_num_peaks) for sp in sps]

    # Get all the peaks
    peaks = []
    for i in range(nsp):
        sp = sps[i]
        peaks = peaks + [(mz, intensity, {i}) for mz, intensity in zip(sp.mz, sp.intensity)]
                    # (peak mz, peak intensity, ids of spectra having a peak at this position)

    # Merge the closest pairs of peaks
    while True:
        peaks.sort()    # Sort by increasing mz
        pair_scores = []

        for i in range(len(peaks)-1):
            mz_diff = peaks[i+1][0] - peaks[i][0]
            if mz_diff < tol: # Always positive because ordered
                pair_scores.append( (mz_diff, (i,i+1)) )

        if len(pair_scores) == 0: # No more peaks can be merged
            # Scale the peaks according to the probability they appear in a spectrum
            # see Frank et al. https://pubs.acs.org/doi/abs/10.1021/pr070361e?casa_token=s05YEJTDKdsAAAAA:EEPc2E5byfAfCZtat1j4r65xOh5vFLtohaP0Zvs5cLuYZxZSN3axNyVBrk7dKzANbB69IFtHeHiX1ACp
            scaled_peaks = []

            for p in peaks:
                prob = len(p[2]) / nsp
                scaled_peaks.append( (p[0], p[1] * (0.95 + 0.05*(1+prob)**5)) )

            return MsmsSpectrum(
                consensus_identifier,
                np.average([sp.precursor_mz for sp in sps]),
                sps[0].precursor_charge,
                [p[0] for p in scaled_peaks],
                [p[1]/nsp for p in scaled_peaks]
            ).filter_intensity(min_intensity, max_num_peaks)

        # Merge the pair having the highest score
        pair_scores.sort(reverse=True)
        i = pair_scores[0][1]   # Peaks indices
        p = peaks[i[0]], peaks[i[1]]
        w = len(p[0][2]), len(p[1][2])
        new_mz = (w[0]*p[0][0] + w[1]*p[1][0]) / (w[0] + w[1])
        new_peak = (new_mz, p[0][1] + p[1][1], p[0][2].union(p[1][2]))

        # Update the list of peaks
        peaks.remove(p[0])
        peaks.remove(p[1])
        peaks.append(new_peak)



### Filter some clusters and export the corresponding spectra

In [ ]:
#mask = (cl_summary['type'] == 'unidentified') & \
#       (cl_summary['prop_maj_tissue'] == 1) & (cl_summary['size'] >= 20)
#print('Number of large unidentified clusters: %d' % (sum(mask),))
#cl_summary[mask].sort_values(by='size', ascending=False)
exp =  [289001,
        382377,
        301107,
        291997,
        417060,
        395555,
        477049,
        458065,
        448727,
        269290,
        549144]

mask = cl_summary['id'].isin(exp)
print(sum(mask))

In [ ]:
import importlib
importlib.reload(ms_io)

sps_consensus_all = []
id_clusters_all = []

limit = 5000
cnt = 0

for index, row in tqdm(cl_summary[mask].iterrows(), total=len(cl_summary[mask])):
    if cnt > limit:
        break

    id = row['id']

    # Get the identifiers of the corresponding spectra
    curr_sps = cluster_labels[cluster_labels['cluster'] == id]
    #print(id, len(curr_sps))

    # Load the bucket containing the spectra
    precursor_charge = curr_sps['precursor_charge'].iloc[0]
    mz_bucket = math.floor(curr_sps['precursor_mz'].iloc[0])

    file_cl = 'consensus_0.1/repr_and_cluster/%d_%s_%d.mgf' % (row['size'], row['maj_tissue'], id)

    #if path.isfile(file_cl):
    #    continue

    #print(curr_sps['identifier'].tolist())
    sps = ms_io.get_one_spectrum_from_pkl(
            '/media/maesk/WD/falcon/PXD000561/spectra',
            precursor_charge, mz_bucket, curr_sps['identifier'].tolist())
    #print(max(sps[0].intensity), max(sps[1].intensity))
    sp_consensus = representative_spectrum(sps, 'consensus:%s:cluster_%d' % (row['maj_tissue'],id,))
    sps_consensus_all.append(sp_consensus)

    #print(sp_consensus.mz)
    #print(sps[0].mz)

    '''fig, ax = plt.subplots(figsize=(12, 6))
    sup.mirror(sp_consensus,
               sps[0],
               ax=ax)
    plt.show()
    plt.close()'''

    # Export the representative + the cluster
    ms_io.write_spectra(file_cl, [sp_consensus] + sps)
    cnt = cnt + 1

# At the end, export all the representative spectra to mgf
ms_io.write_spectra('consensus_0.1/5000_batch_3.mgf', sps_consensus_all)
